<a href="https://colab.research.google.com/github/TheBoy-WhoCode/fake-news-detection/blob/master/TweeetMiner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import tweepy as tw
import datetime
import pandas as pd
print(tw.__version__)

In [0]:
class TweetMiner(object):
  result_limit = 20
  data = []
  api = False
  search_words = "#coronavirus"

  twitter_keys = {
      'consumer_key' : 'XXXXXXXXXXXXXX',
      'consumer_secret' : 'XXXXXXXXXXXX',
      'access_token' : 'XXXXXXXXXXXX',
      'access_token_secret' : 'XXXXXXXXXXXX'
  }

  def __init__(self, keys_dict=twitter_keys, api=api, result_limit=result_limit):
    self.twitter_keys = keys_dict
    
    auth = tw.OAuthHandler(keys_dict['consumer_key'], keys_dict['consumer_secret'])
    auth.set_access_token(keys_dict['access_token'], keys_dict['access_token_secret'])

    self.api = tw.API(auth, wait_on_rate_limit=True)
    self.twitter_keys = keys_dict
    self.result_limit = result_limit
    # self.api.update_status(status='Test')

  def mine_tweets(self, mine_retweets=True, tweet_count=1000):
    data = []
    last_tweet_id = True

    statuses = tw.Cursor(self.api.search,
                             q=self.search_words, 
                             count=self.result_limit,
                             tweet_mode='extended',
                             included_retweets=True,
                             lang="en").items(tweet_count)
    # mined = [[tweet.id, 
    #           tweet.user.name, 
    #           tweet.user.screen_name, 
    #           tweet.full_text,
    #           datetime.datetime.now(),
    #           tweet.created_at,
    #           tweet.favourite_count,
    #           tweet.entities['hashtags'],
    #           tweet.user.statuses_count,
    #           tweet.place,
    #           tweet.source,
    #           tweet.user.verified] for tweet in statuses]    

    for item in statuses:

      mined = {
        'tweet_id': item.id,
        'name': item.user.name,
        'screen_name': item.user.screen_name,
        'retweet_count': item.retweet_count,
        'text': item.full_text,
        'mined_at': datetime.datetime.now(),
        'created_at': item.created_at.strftime("%d-%b-%Y"),
        'favourite_count': item.favorite_count,
        'hashtags': item.entities['hashtags'],
        'urls' : item.entities['urls'],
        'user_mentions' : item.entities['user_mentions'],
        'status_count': item.user.statuses_count,
        'location': item.user.location,
        'source_device': item.source,
        'verfied' : item.user.verified,
        'source_url' : item.source_url
        }                     
      
      try:
        mined['retweet_text'] = item.retweeted_status.full_text
      except:
        mined['retweet_text'] = "None"
      try:
        mined['quote_text'] = item.quoted_status.full_text
        mined['quote_screen_name'] = status.quoted_status.user.screen_name
        # mined['urls'] = item.entities['urls']
        # mined['user_mentions'] = item.entities['user_mentions']
      except:
        mined['quote_text'] = 'None'
        mined['quote_screen_name'] = 'None'
        # mined['urls'] = "None"
        # mined['user_mentions'] = "None"

      data.append(mined)

    return data


In [0]:
miner = TweetMiner(result_limit=20)

In [0]:
mined_tweets = miner.mine_tweets(tweet_count=1800)

In [0]:
print(mined_tweets)

In [0]:
tweet_text = pd.DataFrame(mined_tweets)

In [0]:
tweet_text.to_csv("file_name.csv")

In [0]:
df = pd.read_csv("/content/file_name.csv")
df